## Staph data
http://users-birc.au.dk/cstorm/courses/MLiB_f14/project1.html

In [20]:
import requests
import numpy as np
import pandas as pd
import re

In [2]:
staph_url = ["http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/genome1.fa",
            "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/genome2.fa",
           "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/genome3.fa"]
staph_annot = ["http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/annotation1.fa",
              "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/annotation2.fa"
              "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/annotation3.fa"]

## Load/ Modify Data

In [3]:
def load_file(url,genome=True):
    """input: url for genome, 
    output: pulled data without header and as a single txt string"""
    data = ''.join(requests.get(url).text.split("\n")[1::])
    return data

def segment_genome(data_string,split_size):
    """input: genome data string (data_string) and then size we should split the data (split_size)
    output: array split by specified split_size"""
    chunks = len(data_string)
    data = [data_string[i:i+split_size] for i in range(0,chunks,split_size)]
    return data

def gene_label(annotation, min_gene=10):
    """Input: annotation
    ouput: the label array for data (true if C or R else false for > 50% of line)"""
    size = len(annotation[0])
    print(size)
    label = [True if len(line.strip("C")) < size/2 or len(line.strip("R")) < size/2 else False 
             for line in annotation]
    return label

def gene_label_dir(annotation,direction="C"):
    """Input: annotation and direction is C (forward) unless specified as R (reverse)
    ouput: the label array for data"""
    size = len(annotation[0])
    label = [True if len(line.strip(direction)) < size/2 else False 
             for line in annotation]
    return label

def nucleotide_frequency(seq):
    '''Count the occurrences of characters in "seq".'''
    counts = {'A':0,'C':0,'G':0,'T':0}
    for c in seq:
        counts[c] +=1
    return counts

In [4]:
import re

def ORF_finds(genome):
    starts = [m.start() for m in re.finditer('ATG', genome)]
    return starts

def sequence_list(genome,ends,start):
    sequences = [genome[start:start+end+3] for end in ends[0]]
    sequences += [genome[start:start+end+3] for end in ends[1]] 
    sequences += [genome[start:start+end+3] for end in ends[2]]
    return sequences

def sequence_list2(genome,ends,start,direction):
    frame = start % 3
    sequences = [(start, start+end+3, 'taa', frame, direction) for end in ends[0]]
    sequences += [(start, start+end+3, 'tag', frame, direction) for end in ends[1]] 
    sequences += [(start, start+end+3, 'tga', frame, direction) for end in ends[2]]
    return sequences

def geneList(genome,direction):
    starts = ORF_finds(genome)
    sequences = []
    for start in starts:
        ends = [[m.start() for m in re.finditer(x, data[start:start+2000]) if m.start() % 3 == 0] for x in ["TAA", "TAG", "TGA"]]
        sequences.append(sequence_list2(genome,ends,start,direction))
    return sequences   

def potentialGenes(genome):
    forwards = geneList(genome,'F')
    reversed_genome = reverse_complement(str(genome))
    reverse = geneList(reversed_genome,'R')
    combos = forwards + reverse
    return combos

In [5]:
def reverse_complement(genome):
    """input: genome, output: reverse complement"""
    switch = {"A":"T","G":"C","T":"A","C":"G"}
    rc = ''.join([switch[letter] for letter in genome])[::-1]
    return rc

In [6]:
codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q":["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"Stop":["TAA", "TAG", "TGA"]}

def invert(d):
    return dict( (v,k) for k in d for v in d[k] )

codon_to_aa = invert(codon_map)

def codonToAA(strand):
    aa_seq = ''.join([codon_to_aa(codon) for codon in strand])
    return aa_seq

In [7]:
## amino acid to nucleotide chart http://www.cbs.dtu.dk/courses/27619/codon.html
aa = ["I","L","V","F","M","C","A","G","P","T","S","Y","W","Q","N","H","E","D","K","R","STOP"]
aa_name = ["Isoleucine","Leucine","Valine","Phenylalanine","Methionine","Cysteine","Alanine",
           "Glycine","Proline","Threonine","Serine","Tyrosine","Tryptophan",
           "Glutamine","Asparagine","Histidine","Glutamic_acid", "Aspartic_acid", 
           "Lysine","Arginine","Stop_codons"]
codons = [["ATT", "ATC", "ATA"],["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          ["GTT", "GTC", "GTA", "GTG"],["TTT", "TTC"],["ATG"],["TGT", "TGC"],
          ["GCT", "GCC", "GCA", "GCG"],["GGT", "GGC", "GGA", "GGG"],
          ["CCT", "CCC", "CCA", "CCG"],
          ["ACT", "ACC", "ACA", "ACG"],["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          ["TAT", "TAC"],["TGG"],["CAA", "CAG"],["AAT", "AAC"],["CAT", "CAC"],["GAA", "GAG"],
          ["GAT", "GAC"],["AAA", "AAG"],["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],
          ["TAA", "TAG", "TGA"]]

codon_map = {"I":["ATT", "ATC", "ATA"],
             "L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
             "V":["GTT", "GTC", "GTA", "GTG"],
             "F":["TTT", "TTC"],"M":["ATG"],
             "C":["TGT", "TGC"],
             "A":["GCT", "GCC", "GCA", "GCG"],
             "G":["GGT", "GGC", "GGA", "GGG"],
             "P":["CCT", "CCC", "CCA", "CCG"],
             "T":["ACT", "ACC", "ACA", "ACG"],
             "S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
             "Y":["TAT", "TAC"],
             "W":["TGG"],
             "Q":["CAA", "CAG"],
             "N":["AAT", "AAC"],
             "H":["CAT", "CAC"],
             "E":["GAA", "GAG"],
             "D":["GAT", "GAC"],
             "K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],
             "Stop":["TAA", "TAG", "TGA"]}

codon_properties_map = {}

In [8]:
## Thermofisher aa properties table
from lxml import html
def pull_page(url):
    data = requests.get(url).text
    return data
tag = "table table-bordered table-striped"
tag = "table"
url = "https://www.thermofisher.com/us/en/home/life-science/protein-biology/protein-biology-learning-center/protein-biology-resource-library/pierce-protein-methods/amino-acid-physical-properties.html"

page = html.fromstring(pull_page(url))
tables = page.cssselect(tag)
aa_table = pd.read_html(html.tostring(tables[0]),header=0)[0]


### aa_properties
Aliphatic: Alanine,Isoleucine,Leucine,Valine
Aromatic: Phenylalanine, Tryptophan, Tyrosine
Polar_neutral: Asparagine, Cysteine, Glutamine, Methionine,Serine,Threonine
charged: Aspartic acid, Glutamic acid
    
Properties table
https://www.thermofisher.com/us/en/home/life-science/protein-biology/protein-biology-learning-center/protein-biology-resource-library/pierce-protein-methods/amino-acid-physical-properties.html

http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3456822/pdf/10867_2004_Article_396406.pdf

http://www.proteinstructures.com/Structure/Structure/amino-acids.html

In [17]:
flipped_annot = annot[::-1]

def check_gene_state(loc,annot,strand):
    
    start, end = int(loc[0]), int(loc[1])
    fwd = list(set(annot[start:end+1]))
    bhd = list(set(annot[start - 1:end]))
    if strand == 'F':
        states = list(set(annot[start:end]))
        state = 'C'
    else:
        r_annot = flipped_annot
        states = list(set(r_annot[start:end]))
        state = 'R'
    if len(states) == 1 and states[0] == state and len(fwd) != 1 and len(bhd) != 1:
        return True
    else:
        return False

In [10]:
def build_training_data(annot, data):
    """This finds all the genes, non-coding regions, 
    reverse regions in the genome and returns them as a list
    annot is the annotated genome with c's as forward genes, 
    n's as noncoding regions and r's as genes on the reverse complement strand. 
    data is the genome comprised of a's,t's,g's,c's. 
    Both annot and data are strings"""
    d = []
    i = 0
    c = ''
    start = 0
    while i < len(annot)-1:
        if str(annot[i]) != str(annot[i+1]):
            end = i
            d.append((start,end,str(c),str(annot[i-1])))
            c = ''
            start = i+1
        else:
            c = c + data[i]
        i += 1
    d.append((start,i,str(c),str(annot[i-1])))   
    return d

In [11]:
from itertools import groupby

def find_regions(annot,data):
    """This finds all the genes, non-coding regions, 
    reverse regions in the genome and returns them as a list
    annot is the annotated genome with c's as forward genes, 
    n's as noncoding regions and r's as genes on the reverse complement strand. 
    data is the genome comprised of a's,t's,g's,c's. 
    Both annot and data are strings"""
    stateGroups = groupby(enumerate(annot), lambda value: value[1])
    genome_regions = [(state, [x for x, _ in iterator]) for state, iterator in stateGroups] 
    regions = [(region[0],min(region[1]),max(region[1]),
                data[min(region[1]):max(region[1])+1]) for region in genome_regions]
    return regions

In [12]:
def frameShift(seq,shift):
    """input: seq and shift -- shift not used...."""
    new_seq = ''
    for j, i in enumerate(seq):
        if j % shift == 0:
            new_seq = new_seq + i + ' '
        else: 
            new_seq = new_seq + i
    return new_seq


# fs = [frameShift(seq,3) for seq in gene_df.sequence]

In [13]:
def state(annot):
    """input: states are possible annotations C forward gene,
    N - not a gene, or R for reverse gene"""
    states = list(set(annot))
    return states

def transitions(annot,state):
    """input: determine when transitions occur where annot is annotated 
    and state is the state i'm looking to transition from ouput: there location"""
    trs = list(set(annot))
    transits = {}
    total = len(annot) 

    for transition_state in trs:
        count = len([ts for i,ts in enumerate(annot[:-1]) if str(ts) == state and str(annot[i+1]) == transition_state])
        transits[str(transition_state)] = count/total
    return transits

def emissions(genome,annot,state):
    """input: genome is  a genome, annot is its annotation, 
    state is the state i want to count nucleotide frequency of"""
    ems = list(set(genome))
    
    emissions = {}
    transition_totals = {}

    total = len([letter for letter in annot if str(letter) == state])
    for emission_state in ems:
        count = len([letter for i,letter in enumerate(genome) if 
                     str(letter) == emission_state and str(annot[i])==state])
        emissions[str(emission_state)] = count/total    
    return emissions

def prior_state_emissions(genome,annot):
    """input: """
    state_dictionary = {}
    states = state(annot)
    for s in states:
        if s == 'N':
            prob = 1
        else:
            prob = 0
        em = emissions(genome,annot,s)
        tr = transitions(annot,s)
        state_dictionary[s] = [str(s),prob,em,tr]
    return state_dictionary

In [14]:
# from collections import defaultdict
def aa_find(seq):
    """find aa's in sequence
    input: sequence, 
    output: dictionary of codons in sequence"""
    codon_count = {}
    for aa, codons in codon_map.items():
        codon_count[aa] = 0
        if any(x in seq for x in codons):
            codon_count[aa] += 1
    return codon_count

In [15]:

def is_gene():
    ## for each start in gene list find one with appropriate end and label as true
    ## generate list of genes
    
    ## fill in the empty spots with false
    return False

In [16]:
### Pipeline

## load genome annotation
annot = load_file(staph_annot[0])

## load genome
data = load_file(staph_url[0])

## Built table of sequences for each state in genome
re = find_regions(annot,data)
# re = build_training_data(annot,data)
regions = pd.DataFrame(re)
regions.columns = [["state","start","end","seq"]]
regions.head()
print("Built table of sequences for each state in genome...")
print(regions.head(5))

## Reverse Complement strands found on the Reverse strand
regions.seq = regions.apply(lambda x: reverse_complement(x[3]) if x[0] == "R" else x[3],axis=1)
print("State 'R' sequences were reverse complemented...")

Built table of sequences for each state in genome...
  state  start   end                                                seq
0     N      0   230  TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATGAAAAAT...
1     C    231  1586  ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...
2     N   1587  1740  CATGTGGAAAAGAATATCTTTTATGAAATAGTTATCCACAAGTTGT...
3     C   1741  2877  ATGATTCAATTTTCAATTAATCGCACATTATTTATTCATGCTTTAA...
4     N   2878  2951  GTAAGAAAAAGCTCCCTTTTAGGAGTTTTTTTGTTATTATAAATAT...
State 'R' sequences were reverse complemented...


In [21]:
## Find potential sequences
pot_sequences = potentialGenes(data)
print("Found potential sequences...")

## Create DataFrame
df = pd.DataFrame(list(pd.DataFrame(pot_sequences).unstack().dropna()))
df.columns=[["start","end","end_codon","shift","strand"]]
print("Genome DataFrame Built...")

Found potential sequences...
Genome DataFrame Built...


In [ ]:
## List of C Genes
C_regions = regions[regions.state == 'C']
R_regions = 

In [22]:
## Add length of each sequence
df["len"] = df.apply(lambda x: x[1] - x[0],axis=1) #x[1] - x[0])
print("Calculated sequence lengths...")

Calculated sequence lengths...


In [18]:
## Obtain potential gene sequence  -- TIME SUCK
df["seq"] = df.apply(lambda x: data[x[0]:x[1]],axis=1)
print("Obtained potential sequences...")

## Nucleotide Frequencies for each Potential Gene
res = [[l[1] for l in list(nucleotide_frequency(seg).items())] for seg in list(df.seq)]
frequency_df = pd.DataFrame(res)
frequency_df.columns=["a","c","g","t"]
print("Calculate Nucleotide Frequencies...")

## Combine tables
# 1. reset df index
df.reset_index(inplace=True)
# 2. concat
df = pd.concat([df,frequency_df],axis=1)
print("Combined frequence and gene tables...")

NameError: name 'df' is not defined

In [66]:
## Filter for Genes
    # x[0] - start
    # x[1] - end
    # x[4] - strand
# df["is_gene"] = df.apply(lambda x: check_gene_state([x[0],x[1]],annot, x[4]), axis=1)

In [341]:
y = nsample_df.is_gene
x = nsample_df[["start","end","shift","len","a","c","g","t"]]

In [357]:
nsample_df.is_gene.value_counts()

False    9995
True        5
Name: is_gene, dtype: int64

In [326]:
set(nsample_df.isnull().any(axis=1))

{False}

In [290]:
sample_df["len"] = sample_df.seq.apply(lambda x: len(x))


In [291]:
Nsample_df = sample_df.reindex(np.arange(10000))

In [296]:
set(sample_df.isnull().any(axis=1))

{False}

In [283]:
ex = Nsample_df.seq
# ex.apply(lambda x: len(str(x)))
sample_df.isnull().shape

(10000, 7)

In [361]:
xxx = list(sample_df[sample_df.is_gene == True].seq)

In [362]:
import swalign
# choose your own values here… 2 and -1 are common.
match = 2
mismatch = -1
scoring = swalign.NucleotideScoringMatrix(match, mismatch)

sw = swalign.LocalAlignment(scoring)  # you can also choose gap penalties, etc...
alignment = sw.align(xxx[0],xxx[1])
alignment.dump()

Query:    1 ATG--ATTC--AAT--T----TTCAA-TTAAT--CGCACATTATTTAT--T--C-ATGCTTT--AA-ATACAACTAAACGTGCTATT-AGCACTAAA-AATG-CC--ATTCCTATTCTTTCATCAA--TAA--A--AA---TTGAAGTCACTT--CTACAGGAGTAACTTTAACAGGGTCT-AACGGTC-AAATATCA--ATTGAAAACACTATTCCTGTAAGTA-ATGAAAAT-GCT-GG-T-TTGCTAATTAC-CTCTCCAGGAGCTATTTTA-T--TAGAAGCTA-GT-T-TTTTTATTAATATTATTTCAAGTTTG-CCAGATATTAGTATAA---ATGT--TAAA-GAAATT-G-AA--CAACACCAAGTT-GTTTTAA-CC-AGTGGT-AAATCAGAGAT-T-AC--CTTAAAA--GGA--AAAGATG-TTG--A-CCAGTA-TC-CTCGT-CTA-CA---A-GAAGTATCAACAG-AAAATCCTTTG-ATTTTA-A-AAACAAAATTATTGAAG-TCT---AT-TATT-GCT-GAAACA-GCT-TTTGCAGCCAGTTTACAAGAA-AG-TC-GTCCTA-TTTTAACAGGAGTTCATATTGTATTAAGTAATCATAA-AGATT-T--TAA--AGC-A--GTAGCGA-CTGA-C-TCTC-ATCG-TAT-G-AGCCAACGTTTAATCACT--TTGGACAAT----ACTT--CAG-CAGAT-T------T--TGATGTAGTT-ATTC-CAAGT--AAATCTTTGAGAGAATTTTCAGCAGTATTTACAGATGATATTGAGACCGTTGAGGTATTTTT-CTCACCA-AGCCA---A--ATCTTGTTCAGAAGTGAACACATTT-CTTTTTAT-ACACG-CCTCTTAGAAGG-AAATTATCCCGATA-CAGAC-CGTT--T-A-TT-AATGACAGAATTTGAGACGGAGGT--TGTTTTCAATAC--CC-AATCCCTTCGC-CAC-G

In [308]:
28+13+19+15

75

In [301]:
frequency_df["start"] = sample_df.start

In [304]:
frequency_df.drop(frequency_df.start)

ValueError: labels [ nan  nan  nan ...,  nan  nan  nan] not contained in axis

In [258]:
frequency_df.shape

(10000, 4)

In [259]:
new_df = pd.concat([Nsample_df,frequency_df],axis=1)

In [260]:
new_df.shape

(10000, 11)

In [ ]:
nucleotide_frequency(seq)

In [201]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'char_wb', ngram_range=(1,3), min_df= 1)
d = vectorizer.fit_transform(list(sample_df['seq']))


In [98]:
check_gene_state((37, 1438),annot,'C','F')

False

In [87]:
sample_df.iloc[1].seq

u'ATGAAAAATAGTTGAAAACAATAGCGGTGTCCCCTTAAAATGGCTTTTCCACAGGTTGTGGAGAACCCAAATTAACAGTGTTAATTTATTTTCCACAGGTTGTGGAAAAACTAACTATTATCCATCGTTCTGTGGAAAACTAGAATAGTTTATGGTAGAATAGTTCTAGAATTATCCACAAGAAGGAACCTAGTATGA'

In [ ]:
df["sequence_s"] =  df[['start', 'end']].apply(lambda x: data[x[0]:x[1]], axis=1)

In [39]:
result = [[l[1] for l in list(nucleotide_frequency(seg).items())] for seg in seg_50]
df = pd.DataFrame(result)
df.columns=["a","c","g","t"]
df["gene"] = gene_label(seg_50_an)

gene_df = pd.DataFrame({"label":df.gene,"sequence": seg_50})

50


In [124]:
str(u'x')

'x'

In [122]:
he = prior_state_emissions(data,annot)

In [123]:
he

{u'C': ['C',
  0,
  {'A': 0.3129376364986609,
   'C': 0.18263194813359745,
   'G': 0.20960154402206216,
   'T': 0.2948288713456795},
  {'C': 0.39282114788001343, 'N': 0.0004367210615614748, 'R': 0.0}],
 u'N': ['N',
  1,
  {'A': 0.3190287760527498,
   'C': 0.1789016522921669,
   'G': 0.18693843939846827,
   'T': 0.315131132256615},
  {'C': 0.0004367210615614748,
   'N': 0.2719266092685273,
   'R': 0.000344950257525071}],
 u'R': ['R',
  0,
  {'A': 0.29606142439037003,
   'C': 0.20997386780698055,
   'G': 0.1825552662752494,
   'T': 0.3114094415274},
  {'C': 0.0, 'N': 0.000344950257525071, 'R': 0.3336883603850271}]}

In [ ]:
emi = emissions(data,annot)

In [100]:
trs = transitions(annot)

In [ ]:
# dir(hmm)
from hmm import hmm

In [133]:
he[u'N'][2]

{'A': 0.3190287760527498,
 'C': 0.1789016522921669,
 'G': 0.18693843939846827,
 'T': 0.315131132256615}

In [136]:
NC = hmm.state(he[u'N'][0],he[u'N'][1],he[u'N'][2],he[u'N'][3])
C = hmm.state(he[u'C'][0],he[u'C'][1],he[u'C'][2],he[u'C'][3])
R = hmm.state(he[u'R'][0],he[u'R'][1],he[u'R'][2],he[u'R'][3])
gene_model = hmm.hmm(['A','T','G','C'],[NC,C,R])

In [139]:
gene_model.enumerate('ATGC')

('N', 'C', 'C', 'C'): -6.615035


ValueError: math domain error

In [83]:
s1 = hmm.state(
        'S1',            # name of the state
        0.5,             # probability of being the initial state
        { '1': 0.5,      # probability of emitting a '1' at each visit
          '2': 0.5 },    # probability of emitting a '2' at each visit
        { 'S1': 0.9,     # probability of transitioning to itself
          'S2': 0.1 })   # probability of transitioning to state 'S2'
s2 = hmm.state('S2', 0.5,
        { '1': 0.25, '2': 0.75 },
        { 'S1': 0.8, 'S2': 0.2 })
model = hmm.hmm(['1', '2'],  # all symbols that can be emitted
                [s1, s2])    # all of the states in this HMM

In [ ]:
model = hmm.hmm(['A', 'C','T','G'],  # all symbols that can be emitted
                [s1, s2])

In [84]:
model.enumerate('222')

('S2', 'S2', 'S2'): -2.073786
('S2', 'S2', 'S1'): -1.647817
('S2', 'S1', 'S2'): -1.948847
('S2', 'S1', 'S1'): -1.170696
('S1', 'S2', 'S2'): -2.550907
('S1', 'S2', 'S1'): -2.124939
('S1', 'S1', 'S2'): -2.073786
('S1', 'S1', 'S1'): -1.295635
BEST: ('S2', 'S1', 'S1'): -1.170696


In [45]:
training_data = build_training_data(annot,data)

In [58]:
import hmm

In [57]:
tabul = hmm(data, annot)

TypeError: 'module' object is not callable

In [54]:
train_hmm(training_data)

NameError: global name 'state' is not defined

In [ ]:
emissions(data,annot)

In [ ]:
set(data)

In [ ]:
# two_state_transitions(annot,1)
two_state_emissions(data,annot,1)
# str(annot[0]) == "N"

In [ ]:
reverse_complement("aaatttgggcca")

In [ ]:
df["sequence"] = gene_df.sequence
df["start"] = gene_df.sequence.apply(lambda seq: "ATG" in seq)
df["stop"] = gene_df.sequence.apply(lambda seq: any(x in seq for x in ["TAA", "TAG", "TGA"]))
# df.iloc[0].sequence

In [ ]:
# from collections import defaultdict
def aa_find(seq):
    codon_count = {}
    for aa, codons in codon_map.items():
        codon_count[aa] = 0
        if any(x in seq for x in codons):
            codon_count[aa] += 1
    return codon_count

In [ ]:
df["dict"] = gene_df.sequence.apply(lambda seq: aa_find(seq)) 

In [ ]:
aa_counts = pd.DataFrame(list(df["dict"]))

In [ ]:
# aa_counts[["a"],["c"],["g"],["t"],["gene"]] = [[df.a],[df.c],[df.g],[df.t],[df.gene]]
# aa_counts[["a"]]
aa_counts["a"] = df.a
aa_counts["c"] = df.c
aa_counts["g"] = df.g
aa_counts["t"] = df.t
aa_counts["gene"] = df.gene


In [ ]:
df.shape

In [ ]:
a = []
a.append(aa_find(df.iloc[1].sequence).values())
a.append(aa_find(df.iloc[2].sequence).values())
a.append(aa_find(df.iloc[3].sequence).values())
pd.DataFrame(a)

In [ ]:
[a[1] for a in aa_find(df.iloc[1].sequence).items()]

In [ ]:
aa_find(df.iloc[1].sequence).items()

In [ ]:
codon_to_aa.items()
codon_map.items()


## Feature Development

In [ ]:
# ngrams
- 3 grams, which frame?, relationship between subsequent sequence fragments.
... am I next to a fragment

In [ ]:
"atg" in "gowijgoijatgsoijdfo"

In [ ]:
gene_df.sequence[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'char_wb', ngram_range=(1,3), min_df= 1)
d = vectorizer.fit_transform(list(gene_df['sequence']))

In [ ]:
def frameShift(seq,shift):
    new_seq = ''
    for j, i in enumerate(seq):
        if j % shift == 0:
            new_seq = new_seq + i + ' '
        else: 
            new_seq = new_seq + i
    return new_seq


fs = [frameShift(seq,3) for seq in gene_df.sequence]

In [ ]:
fs[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'char_wb', ngram_range=(3,3), min_df= 1)
d = vectorizer.fit_transform(fs)

In [ ]:
fs[0]

In [ ]:
x = vectorizer.vocabulary_.copy()
my_dict2 = dict((y,str(x)) for x,y in x.iteritems())

In [ ]:
my_dict2 = dict((y,str(x)) for x,y in x.iteritems())

In [ ]:
my_dict2

In [ ]:
nx = [x.pop(val) for val in vectorizer.vocabulary_ if len(val.strip()) != 3]


In [ ]:
gg = list(d)

In [ ]:
len(gg)

In [ ]:
len(d.indptr)
len(d.data)

In [ ]:
test = gg[0]

In [ ]:
import pandas as pd

l = []
for i in range(len(gg)):
    doc = gg[i]
    l.append(pd.DataFrame({"doc":i,"index":doc.indices,"count":doc.data}))
dum = pd.concat(l)
# df2 = pd.DataFrame({"index":doc.indices,"count":doc.data})
# df2


In [ ]:
len(set(dum.doc))

In [ ]:
dum.index

In [ ]:
x = vectorizer.vocabulary_.copy()
my_dict2 = dict((y,str(x)) for x,y in x.iteritems())
dum["codon"] = dum['index'].apply(lambda ind: my_dict2[ind])
codonCount = dum[dum.codon.apply(lambda v: len(v.strip())) == 3]

In [ ]:
codonCount = dum[dum.codon.apply(lambda v: len(v.strip())) == 3]
# dum.codon.apply(lambda v: len(v.strip()))

In [ ]:
codonCount

In [ ]:
ngrams = list(vectorizer.get_feature_names())
[gram.strip() for gram in ngrams if len(gram.strip()) == 3]




In [ ]:
df

In [ ]:
codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q":["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"Stop":["TAA", "TAG", "TGA"]}
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )
codon_to_aa = invert(codon_map)

def codonToAA(strand):
    aa_seq = ''.join([codon_to_aa(codon) for codon in strand])
    return aa_seq

In [ ]:
codon_to_aa

#e

In [ ]:
def codonToAA(strand):
    aa_seq = ''.join([codon_to_aa(codon) for codon in strand])
    return aa_seq


In [ ]:
new_seq

In [ ]:
ngrams = list(vectorizer.get_feature_names())
ngrams


In [ ]:
## amino acid to nucleotide chart http://www.cbs.dtu.dk/courses/27619/codon.html
aa = ["I","L","V","F","M","C","A","G","P","T","S","Y","W","Q","N","H","E","D","K","R","STOP"]
aa_name = ["Isoleucine","Leucine","Valine","Phenylalanine","Methionine","Cysteine","Alanine",
           "Glycine","Proline","Threonine","Serine","Tyrosine","Tryptophan",
           "Glutamine","Asparagine","Histidine","Glutamic_acid", "Aspartic_acid", 
           "Lysine","Arginine","Stop_codons"]
codons = [["ATT", "ATC", "ATA"],["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          ["GTT", "GTC", "GTA", "GTG"],["TTT", "TTC"],["ATG"],["TGT", "TGC"],
          ["GCT", "GCC", "GCA", "GCG"],["GGT", "GGC", "GGA", "GGG"],
          ["CCT", "CCC", "CCA", "CCG"],
          ["ACT", "ACC", "ACA", "ACG"],["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          ["TAT", "TAC"],["TGG"],["CAA", "CAG"],["AAT", "AAC"],["CAT", "CAC"],["GAA", "GAG"],
          ["GAT", "GAC"],["AAA", "AAG"],["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],
          ["TAA", "TAG", "TGA"]]

codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q"["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"Stop":["TAA", "TAG", "TGA"]}
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )
# aa_properties
Aliphatic: Alanine,Isoleucine,Leucine,Valine
Aromatic: Phenylalanine, Tryptophan, Tyrosine
Polar_neutral: Asparagine, Cysteine, Glutamine, Methionine,Serine,Threonine
charged: Aspartic acid, Glutamic acid
    
Properties table
https://www.thermofisher.com/us/en/home/life-science/protein-biology/protein-biology-learning-center/protein-biology-resource-library/pierce-protein-methods/amino-acid-physical-properties.html

http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3456822/pdf/10867_2004_Article_396406.pdf

http://www.proteinstructures.com/Structure/Structure/amino-acids.html

In [ ]:
codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q":["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"Stop":["TAA", "TAG", "TGA"]}
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )
codon_to_aa = invert(codon_map)

In [ ]:
codon_to_aa

## Model testing

In [339]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, classification_report


from sklearn.learning_curve import learning_curve

from sklearn.cross_validation import train_test_split, KFold, cross_val_score

In [ ]:
# x = df[["a","c","g","t","start","stop"]]
y = df.gene
x = aa_counts.drop('gene', 1)

In [351]:
y = y.apply(lambda x: int(x))

In [352]:
algorithms = [RandomForestClassifier(), DecisionTreeClassifier(), GaussianNB(), 
              SVC(), KNeighborsClassifier(n_neighbors=6), LogisticRegression()]

for algo in algorithms:
    accuracy = cross_val_score(algo,x,y)
    print("{:s} Accuracy Score : {:f}".format(str(algo).split('(', 1)[0],accuracy.mean()))

RandomForestClassifier Accuracy Score : 0.999500
DecisionTreeClassifier Accuracy Score : 0.669966
GaussianNB Accuracy Score : 0.996001
SVC Accuracy Score : 0.999500
KNeighborsClassifier Accuracy Score : 0.999500
LogisticRegression Accuracy Score : 0.996901


In [52]:
import hmm